In [1]:
from udma import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cb=UDMA_CLASS("140.105.17.180", 7)
# cb=UDMA_CLASS("192.168.1.10", 7)
cb.connect()
cb.log(0)

Comblock Registers

|Register | Connection |
| ----- | ---- |
|reg0_o | Config_Res_intan_A|
|reg1_o | Config_Res_intan_B|
|reg2_o | Config_Res_intan_C|
|reg3_o | Config_Res_intan_D|
|reg4_o | Read_intan(3:0)|
|reg5_o | Sys_reset (active low)|
|reg6_o | **Combintan_w_header**/nsamples |
|reg7_o | **Combintan_w_header**/sys_en |





In [3]:
def hicce_init(test_mode=0):
    # Select comblock
    cb.select_comblock(0)
    # Reset HiCCE
    cb.write_reg(5, 0) #RESET ON
    
    #Disable Acquisiton 
    cb.write_reg(4, 0)
    cb.write_reg(7, 0)

    # FIFO Clear
    cb.select_comblock(0)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(1)
    cb.write_reg(17, 1)
    cb.write_reg(17, 0)
    cb.select_comblock(0)

    # Configure INTANT in Sequential mode (Reset Counter)
    cb.write_reg(0, 0x1c02+test_mode)
    cb.write_reg(1, 0x1c02+test_mode)
    cb.write_reg(2, 0x1c02+test_mode)
    cb.write_reg(3, 0x1c02+test_mode)
    # Configure INTANT (Cyclic Read)
    cb.write_reg(0, 0x1c26+test_mode)
    cb.write_reg(1, 0x1c26+test_mode)
    cb.write_reg(2, 0x1c26+test_mode)
    cb.write_reg(3, 0x1c26+test_mode)

def enable_acquisition(NSamples=100):
    cb.select_comblock(0)
    #Set number of samples per package
    cb.write_reg(6, 32*NSamples)

    #Enable acquisition
    cb.write_reg(4, 15) ##Enabling INTAN readout (legacy)
    cb.write_reg(5, 1) ##Enabling HICCE Driver
    cb.write_reg(7, 1) #Enable FIFOs
    
    

def read_channels(Ncomblock=0, NSamp=100, TO=1000):
    cb.select_comblock(Ncomblock)
    wordsInFifo=cb.read_reg(34)[1][0]>>16
    #Wait until FIFO have enough samples to read.
    wd=0
    while wordsInFifo<(32*NSamp+4):
        wordsInFifo_new=cb.read_reg(34)[1][0]>>16
        if wordsInFifo_new == wordsInFifo:
            wd+=1
            if wd==TO: #if the value doesn't change after TO iterations return error
                return -1
        else:
            wordsInFifo=wordsInFifo_new
            wd=0
    if NSamp==1:
        dpack=cb.read_hicce((32*NSamp)+5)[1]
    else:
        dpack=cb.read_hicce((32*NSamp)+4)[1]
    return dpack

def decode(dpack):
    head=dpack[0]
    CEN=(head>>16) & 0xff
    SBT=(head>>8) & 0xff
    SAT=head & 0xff
    TS=dpack[1]<<32 | dpack[2]
    TAIL=dpack[-1]
    CENT=(TAIL>>16) & 0xff
    FLAGS=TAIL & 0xffff

    CH=[]
    for i in range(64):
        CH.append([])
    
    for i in range(len(dpack[3:-1])):
        d=dpack[i]
        CH[(i%32)].append((d>>16))
        CH[(i%32)+32].append((d&0xffff))
    
    return CH, TS, (CEN, CENT, SBT, SAT, FLAGS)

In [11]:
cb.s.recv(500)

KeyboardInterrupt: 

In [13]:
hicce_init(test_mode=0)
enable_acquisition(20)

In [12]:
cb.select_comblock(0)
print("CB FIFO Size",cb.read_reg(34)[1][0]>>16)

cb.select_comblock(1)
print("CB FIFO Size", cb.read_reg(34)[1][0]>>16)


CB FIFO Size 32768
CB FIFO Size 32767


In [9]:
hicce_init(test_mode=1)
enable_acquisition(20)
sampling_time=4e-9
NofIterations=50
ch=[]
tab=[]
tcd=[]
for i in range(128):
    ch.append([])
for i in range(NofIterations):
    ab_raw=read_channels(0,20)
    cd_raw=read_channels(1,20)
    CHAB, TSAB, FLAGS=decode(ab_raw)
    CHCD, TSCD, FLAGS=decode(cd_raw)
    tabtemp=[TSAB*sampling_time + i * sampling_time for i in range(len(CHAB[0]))]
    tcdtemp=[TSCD*sampling_time + i * sampling_time for i in range(len(CHCD[0]))]
    for i in range(64):
        ch[i]=ch[i]+CHAB[i]
        ch[i%64+64]=ch[i%64+64]+CHCD[i]
    tab=tab+tabtemp
    tcd=tcd+tcdtemp
    


In [10]:
%matplotlib widget
plt.plot(np.array(ch[0], dtype=np.int16))

/tmp/ipykernel_2433919/1493929050.py:2: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 40061 to int16 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  plt.plot(np.array(ch[0], dtype=np.int16))
/tmp/ipykernel_2433919/1493929050.py:2: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 48061 to int16 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)`
will give the desired result (the cast overflows).
  plt.plot(np.array(ch[0], dtype=np.int16))
/tmp/ipykernel_2433919/1493929050.py:2: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 56061 to int16 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dt

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
len(ch[0])

In [ ]:
# hicce_init()
# enable_acquisition(20)
# ch0=[]
# for i in range(50):
#     ab_raw=read_channels(0,20)
#     CHAB, TSAB, FLAGS=decode(ab_raw)
#     ch0=ch0+CHAB[30]


In [ ]:
cb.write_reg(5,0)

In [ ]:
hicce_init()

In [ ]:
hicce_init(test_mode=1)
enable_acquisition(20)
ch=[]
for i in range(64):
    ch.append([])
for i in range(1):
    ab_raw=read_channels(0,20)
    cd_raw=read_channels(1,20)
    CHAB, TSAB, FLAGS=decode(ab_raw)
    for i in range(64):
        ch[i]=ch[i]+CHAB[i]

In [ ]:
%matplotlib widget
plt.plot(ch[2])


In [ ]:
ab_raw=read_channels(0,20)
ab=decode(ab_raw)
cd=decode(read_channels(1, 20))

In [ ]:
np.int16(ab[0][2][2])

In [ ]:
ab=decode(read_channels(0, 20))
cd=decode(read_channels(1, 20))

In [ ]:
for i in range(len(ab[0])):    
    print(len(ab[0][i]))
    # print(len(cd[0][i]))

In [ ]:
for i in range(len(ab)):
    if ab[i]>>24==0x04:
        print(i)
print(len(ab))

In [ ]:
hicce_init()
enable_acquisition(3200)

In [ ]:
ab,cd=read_channels(1, 3200)
DAB=decode(ab)
DCD=decode(cd)
#Timestamp and Channels 0-64 from AB
TSAB=DAB[3]
CHAB=DAB[4]
#Timestamp and Channels 0-64 from CD
TSCD=DCD[3]
CHCD=DCD[4]

plt.plot(CHAB[0])

In [ ]:
print(CHAB[0])

In [ ]:
plt.plot(CHAB)
plt.plot(CHCD)

In [ ]:
import pylsl
pylsl.local_clock()